In [2]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import cv2
from cifar10_models import mobilenetv2
from torchvision import transforms, datasets
from torch.utils.data import Subset, DataLoader
import matplotlib.pyplot as plt

In [3]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [4]:

Net = mobilenetv2.mobilenet_v2(pretrained=True)

In [5]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
])

In [ ]:
#for param in Net.parameters():
#    param.requires_grad = False

In [6]:
class RegressionModel(nn.Module):
    def __init__(self, pretrainedModel):
        super().__init__()
        self.features = pretrainedModel.features
        
        self.flatten = nn.Flatten()
        
        self.Dense1 = nn.Linear(20480, 32)

        self.outLayer1 = nn.Linear(32, 4)
        self.outLayer2 = nn.Linear(32, 4)
        self.outLayer3 = nn.Linear(32, 4)
        self.outLayer4 = nn.Linear(32, 4)
    
    def forward(self, x):
        x = self.features(x)
        x = self.Dense1(self.flatten(x))

        out1 = self.outLayer1(x)
        out2 = self.outLayer2(x)
        out3 = self.outLayer3(x)
        out4 = self.outLayer4(x)

        return out1, out2, out3, out4


In [7]:
dataset = datasets.CIFAR10('../data-unversioned/p1ch7/', train=True, transform=preprocess, download=False)
imgs = os.listdir('data/training')
imgs.sort()
indices = [int(name[0:5]) for name in imgs]
my_subset = Subset(dataset, indices) #create subset based on indices
train_loader = DataLoader(my_subset, batch_size=1, shuffle=False) #shuffle false so we can use correct stuff

In [12]:
labels = torch.from_numpy(np.load('data/labels.npy')).float()
labels = labels.to(device=device)

In [ ]:
labels.shape

In [ ]:
#plt.imshow(myset[1].permute(1, 2, 0))

In [9]:
model = RegressionModel(Net).to(device=device)


In [16]:
img = cv2.imread('data/attacks/00000_input_img.png')
imgT = torch.tensor(img)
imgT = imgT.view(3, 32 , 32).to(device=device)
imgT = torch.unsqueeze(imgT, 0).float()
out1, out2, out3, out4 = model(imgT)
out1[0].shape, out2.shape

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same

In [13]:
loss = nn.MSELoss()

In [14]:
optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [15]:
loss(out1[0], labels[1][0])

labels[1][0].dtype, out1[0].dtype

NameError: name 'out1' is not defined

In [17]:
def training_loop(n_epochs, optimizer, model, loss_fn, loader, labels):    
    for epoch in range(1, n_epochs+1):
        loss_train = 0.0
        idx = 0
        for img_batch, _ in loader:
            
            img_batch = img_batch.to(device=device)
            out1, out2, out3, out4 = model(img_batch)
            loss1 = loss_fn(out1[0], labels[idx][0])
            loss2 = loss_fn(out2[0], labels[idx][1])
            loss3 = loss_fn(out3[0], labels[idx][2])
            loss4 = loss_fn(out4[0], labels[idx][3])

            loss_total = loss1 + loss2 + loss3 + loss4 #accumulate loss

            optimizer.zero_grad()

            loss_total.backward()
            optimizer.step()

            idx += 1
            
            if idx % 100 == 0:
                print(f'step {idx} is the current iteration')
            
        if epoch == 1 or epoch % 2 == 0:
            print(f'At {epoch}, the training loss is {loss_total}')
            

In [18]:
training_loop(n_epochs=4, optimizer=optimizer, model=model, loss_fn=loss, loader=train_loader, labels=labels)

step 100 is the current iteration
step 200 is the current iteration
step 300 is the current iteration
step 400 is the current iteration
step 500 is the current iteration
step 600 is the current iteration
step 700 is the current iteration
step 800 is the current iteration
step 900 is the current iteration
step 1000 is the current iteration
step 1100 is the current iteration
step 1200 is the current iteration
step 1300 is the current iteration
step 1400 is the current iteration
step 1500 is the current iteration
step 1600 is the current iteration
step 1700 is the current iteration
step 1800 is the current iteration
step 1900 is the current iteration
step 2000 is the current iteration
step 2100 is the current iteration
step 2200 is the current iteration
step 2300 is the current iteration
step 2400 is the current iteration
step 2500 is the current iteration
step 2600 is the current iteration
step 2700 is the current iteration
step 2800 is the current iteration
step 2900 is the current iter

In [22]:
asd = my_subset[0][0].to(device=device)

In [24]:
model(torch.unsqueeze(asd, 0))

(tensor([[14.3258, 14.8892,  2.7326,  2.7266]], device='cuda:0',
        grad_fn=<AddmmBackward>),
 tensor([[12.7764, 13.5166,  3.9104,  3.8807]], device='cuda:0',
        grad_fn=<AddmmBackward>),
 tensor([[14.4563, 15.2508,  3.1944,  3.6099]], device='cuda:0',
        grad_fn=<AddmmBackward>),
 tensor([[15.3302, 13.4749,  4.2027,  4.3553]], device='cuda:0',
        grad_fn=<AddmmBackward>))

In [25]:
labels[0]

tensor([[29.3383,  5.3111,  0.9465,  5.0000],
        [ 5.0803, 13.2412,  4.5605,  2.1632],
        [21.7605, 10.3194,  4.6590,  5.0000],
        [13.8442, 19.0300,  5.0000,  5.0000]], device='cuda:0')